In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

import mne
import json



In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")




thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import STUDY_DATA_FOLDER
from predicament.utils.config import E4_LOCAL_DIRPATHS
from predicament.utils.config import E4_CSV_FILES

from predicament.data.timeseries import load_E4_csv_data
from predicament.data.timeseries import merge_E4_csv_data
from predicament.data.timeseries import create_participant_data_E4_only
from predicament.data.timeseries import create_participant_data

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition


In [4]:

E4_full_dirpaths = {
    part_ID: os.path.join(STUDY_DATA_FOLDER,local_path)
        for part_ID, local_path in E4_LOCAL_DIRPATHS.items() }
E4_full_dirpaths

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

In [5]:
E4_CSV_FILES

['TEMP.csv', 'EDA.csv', 'BVP.csv', 'ACC.csv', 'IBI.csv', 'HR.csv']

In [6]:
full_dirpath = E4_full_dirpaths['VG_01']

csv_data = load_E4_csv_data(full_dirpath, csv_files=E4_CSV_FILES)
for k, v in csv_data.items():
    print(f"csv_fname = {k}")
    print(f"start_time = {v['start_time']}")
    print(f"sample_rate = {v['sample_rate']}")
    print(f"timeseries.shape = {v['timeseries'].shape}")
    print(f"timeseries = {v['timeseries']}")
    print()

csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv
csv_fname = TEMP.csv
start_time = 1631112276
sample_rate = 4
timeseries.shape = (1, 14784)
timeseries = [[33.21 33.21 33.21 ... 32.55 32.55 32.55]]

csv_fname = EDA.csv
start_time = 1631112276
sample_rate = 4
timeseries.shape = (1, 14784)
timeseries = [[0.       0.005125 0.002563 ... 0.007688 0.005125 0.007688]]

csv_fname = BVP.csv
start_time = 1631112276
sample_rate = 64
timeseries.shape = (1, 236533)
timeseries = [[-0.   -0.   -0.   ... 27.15 25.4  23.94]]

csv_fname = ACC.csv
start_time = 1631112276
sample_rate = 32
timeseries.shape = (3, 118272)
timeseries = [[-2.  1. -7. ... 23. 22. 22.]
 [25. 31. 31. ... 26. 26. 26.]
 [56. 53. 55. ... 52. 52. 52.]]

csv_fname = HR.csv
start_time = 1631112286
sample_rate = 1
timeseries.shape = (1, 3686)
timeseries = [[67.   67.   93.   ... 74.32 74.63 74.93]]



In [7]:

data, channel_names, sample_rate, std_start_time = merge_E4_csv_data(csv_data, csv_files=E4_CSV_FILES)
print(f"std_start_time = {std_start_time}")
print(f"channel_names = {channel_names}")
print(f"sample_rate = {sample_rate}")
print(f"data.shape = {data.shape}")
print(f"data = {data}")

std_start_time = 1631112276
channel_names = ['temp', 'eda', 'bvp', 'acc0', 'acc1', 'acc2', 'hr']
sample_rate = 64
data.shape = (7, 235264)
data = [[ 3.32100000e+01  3.32391567e+01  3.32637518e+01 ...  3.41289946e+01
   3.40908906e+01  3.40578398e+01]
 [-1.04582048e-15  1.87466145e-04  4.57271777e-04 ...  3.41908935e-04
   1.55837801e-04  4.14152420e-05]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 ...  2.49730000e+02
   2.99870000e+02  3.46430000e+02]
 ...
 [ 2.50000000e+01  2.66135420e+01  3.10000000e+01 ...  6.27237733e+01
   5.70000000e+01  4.03789309e+01]
 [ 5.60000000e+01  5.49431259e+01  5.30000000e+01 ... -4.10299957e+01
  -4.40000000e+01 -4.50049822e+01]
 [ 8.82700000e+01  8.82926913e+01  8.83153148e+01 ...  6.76226974e+01
   6.74146034e+01  6.72069481e+01]]


In [8]:
all_participants_data = create_participant_data_E4_only()


VG_01:setup is valid but timestamps have issues. What does valid mean?
VG_01:baseline is valid but timestamps have issues. What does valid mean?
VG_06:takeoff_EEG is valid but timestamps have issues. What does valid mean?
VG_07:takeoff_EEG is valid but timestamps have issues. What does valid mean?
VG_08:family_inter is valid but timestamps have issues. What does valid mean?
VG_10:setup is valid but timestamps have issues. What does valid mean?
VG_10:takeoff_EEG is valid but timestamps have issues. What does valid mean?
VH_01:family_inter is valid but timestamps have issues. What does valid mean?


  9%|███████████▎                                                                                                                | 1/11 [00:00<00:01,  8.08it/s]

loading directory: ../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/
csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv
Successfully loaded E4 data for VG_01 .
loading directory: ../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/
csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv


 27%|█████████████████████████████████▊                                                                                          | 3/11 [00:00<00:01,  7.85it/s]

Successfully loaded E4 data for VG_03 .
loading directory: ../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/
csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv
Successfully loaded E4 data for VG_05 .
loading directory: ../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/
csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv


...

Successfully loaded E4 data for VH_01 .
loading directory: ../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/
csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv


 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 10/11 [00:04<00:00,  1.05it/s]

Successfully loaded E4 data for VH_02 .
loading directory: ../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/
csv_fname = TEMP.csv
csv_fname = EDA.csv
csv_fname = BVP.csv
csv_fname = ACC.csv
csv_fname = HR.csv


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:05<00:00,  2.20it/s]

Successfully loaded E4 data for VH_03 .


## Data description

From the file '/CARE_HOME_DATA/VG01/E4_8921_15_44/info.txt':

.csv files in this archive are in the following format:
The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.

TEMP.csv
Data from temperature sensor expressed degrees on the Celsius (°C) scale.

EDA.csv
Data from the electrodermal activity sensor expressed as microsiemens (μS).

BVP.csv
Data from photoplethysmograph.

ACC.csv
Data from 3-axis accelerometer sensor. The accelerometer is configured to measure acceleration in the range [-2g, 2g]. Therefore the unit in this file is 1/64g.
Data from x, y, and z axis are respectively in first, second, and third column.

IBI.csv
Time between individuals heart beats extracted from the BVP signal.
No sample rate is needed for this file.
The first column is the time (respect to the initial time) of the detected inter-beat interval expressed in seconds (s).
The second column is the duration in seconds (s) of the detected inter-beat interval (i.e., the distance in seconds from the previous beat).

HR.csv
Average heart rate extracted from the BVP signal.The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.


tags.csv
Event mark times.
Each row corresponds to a physical button press on the device; the same time as the status LED is first illuminated.
The time is expressed as a unix timestamp in UTC and it is synchronized with initial time of the session indicated in the related data files from the corresponding session.

In [13]:
all_participants_data['VG_01'].event_details.events_info

{'setup': {'start': 1631112720.0, 'end': None, 'valid': True},
 'baseline': {'start': 1631112720.0, 'end': None, 'valid': True},
 'exper_video': {'start': 1631113320.0, 'end': 1631113800.0, 'valid': True},
 'wildlife_video': {'start': 1631112960.0, 'end': 1631113260.0, 'valid': True},
 'familiar_music': {'start': 1631114160.0, 'end': 1631114460.0, 'valid': True},
 'tchaikovsky': {'start': 1631113860.0, 'end': 1631114160.0, 'valid': True},
 'family_inter': {'start': 1631114520.0, 'end': 1631114820.0, 'valid': True}}

In [16]:
all_participants_data['VG_01'].event_details.exp_start_time

1631112276